In [19]:
import warnings
warnings.filterwarnings('ignore')

import importlib.util
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler

In [21]:
# specify the module that needs to be
# imported relative to the path of the
# module
spec = importlib.util.spec_from_file_location("loadTrainTestPostedWaitTimes", "../src/data/loadTrainTestData.py")

# creates a new module based on spec
loadTrainPosted = importlib.util.module_from_spec(spec)

# executes the module in its own namespace
# when a module is imported or reloaded.
spec.loader.exec_module(loadTrainPosted)

X_train, X_test, y_train, y_test = loadTrainPosted.loadTrainTestPostedWaitTimes()

In [27]:
X_train["MONTHOFYEAR"] = X_train["date"].dt.month.astype("Int8")
X_train["YEAR"] = X_train["date"].dt.year.astype("Int16")
X_train["DAYOFYEAR"] = X_train["date"].dt.dayofyear.astype("Int16")
X_train["HOUROFDAY"] = X_train["datetime"].dt.hour.astype("Int8")

X_train_encoded = X_train.drop(columns=['date', 'datetime'])

MemoryError: Unable to allocate 981. MiB for an array with shape (40, 3213787) and data type float64

In [ ]:
scaler = MinMaxScaler()

# print(dict(X_train_encoded.dtypes))
linRidge = Ridge(alpha=20.0).fit(X_train_encoded, y_train)


# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# linridge = Ridge(alpha=20.0).fit(X_train_scaled, y_train)

In [ ]:
print('Crime dataset')
print('ridge regression linear model intercept: {}'
     .format(linRidge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linRidge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linRidge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linRidge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linRidge.coef_ != 0)))

In [50]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
import json


with open("../data/interim/dtypes.json") as json_file:
    dtypes = json.load(json_file)

rideDataLst = []
for year in range(2015, 2022):
    print(year)
    rideData = pd.read_csv(f"../data/interim/RideData{year}Weather.csv", dtype=dtypes, compression='gzip')
    rideDataLst.append(rideData)

2015
2016
2017
2018
2019
2020
2021


In [43]:
X = pd.concat(rideDataLst).drop(columns=["SACTMIN", "SPOSTMIN"])
X_num = X.select_dtypes(include=np.number).reset_index(drop=True)

In [45]:
print(type(X_num))

<class 'pandas.core.frame.DataFrame'>


In [56]:
threshold = 0

X_num = X.select_dtypes(include=np.number).reset_index(drop=True)

print(f"BEFORE THRESHOLD {threshold}: ", X_num.shape)

var_thr = VarianceThreshold(threshold = 0.05) #Removing both constant and quasi-constant
var_thr.fit(X_num)

var_thr.get_support()

concol = [column for column in X_num.columns 
          if column not in X_num.columns[var_thr.get_support()]]

X.drop(concol,axis=1)

for features in concol:
    print(features)

BEFORE THRESHOLD 0:  (1616601, 135)
Ride_type_scary
Age_interest_kids
WDWevent
WDWrace
AKevent
AKEMHEVE
AKEMHEYEST
AKEMHETOM
WEATHER_WDWPRECIP
MKFIREWK
EPFIREWK
HSPRDDAY
HSFIREWKS
AKPRDDAY
new_case
Source Element
Weather Type
Weather Type Observation
Weather Code Quality Code


In [59]:
X["AKEMHEVE"].value_counts()

0    1616601
Name: AKEMHEVE, dtype: Int64